In [1]:
%load_ext autoreload
%autoreload 2

import torch
import yaml
import sys
from torchmetrics import MetricCollection
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from tqdm import tqdm
from pathlib import Path
from pytorch_lightning import seed_everything
sys.path.append('fbsource/fbcode/scripts/psarlin/')
from maploc.data.loader_mapillary import MapillaryDataModule
from maploc.module import GenericModule, find_best_checkpoint
from maploc.utils.viz_2d import plot_images, plot_keypoints, features_to_RGB, save_plot, add_text
from maploc.utils.viz_localization import likelihood_overlay, plot_pose, plot_dense_rotations
from maploc.osm.viz import Colormap, plot_nodes
from maploc.models.metrics import Location2DError, AngleError
from maploc.test import evaluate_single_image
torch.set_grad_enabled(False);
plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
conf = OmegaConf.load('fbsource/fbcode/scripts/psarlin/maploc/conf/data_mapillary.yaml')
conf = OmegaConf.merge(conf, OmegaConf.create(yaml.full_load("""
local_dir: "./data/mapillary_dumps_v2/"
# local_dir: "./devvm1417.cln0.facebook.com/data/mapillary_dumps_v2/"
dump_dir: ${.local_dir}
scenes:
    - sanfrancisco_soma
    - sanfrancisco_hayes
    - amsterdam
    - berlin
    - lemans
    - montrouge
    - toulouse
    - nantes
    - vilnius
    - avignon
    - helsinki
    - milan
    # - newyork_hoboken
    # - metropolis_train
# split: None
return_gps: true
val: {batch_size: 1, num_workers: 3}
train: ${.val}
random: false
augmentation: {rot90: false, flip: false}
""")))
if not Path(conf.local_dir).exists():
    conf.local_dir = str(Path("devvm1417.cln0.facebook.com", conf.local_dir))
    assert Path(conf.local_dir).exists()
OmegaConf.resolve(conf)
dataset = MapillaryDataModule(conf)
dataset.prepare_data()
dataset.setup()

In [3]:
exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64-prior"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64"

root = "manifold://psarlin/tree/maploc/experiments"
path = f'{root}/{exper}/last.ckpt'
print(path)
cfg = {}
cfg = {'model': {"num_rotations": 128}}
model = GenericModule.load_from_checkpoint(path, strict=True, find_best=True, cfg=cfg)
model.eval()
if torch.cuda.is_available():
    model = model.cuda(1)
exper += '_nrot128'

In [ ]:
from maploc.osm.parser import Groups
thresholds = [1, 2, 5, 10]
num = 500

results = {}
seed_everything(0)
results["full"] = evaluate_single_image(
    dataset.dataloader("val", shuffle=True), model, thresholds, mask_index=None, num=num)

for j, groups in enumerate([Groups.areas, Groups.ways, Groups.nodes]):
    for i, label in enumerate(groups):
        print(label)
        index = (j, i)
        seed_everything(0)
        results[label] = evaluate_single_image(
            dataset.dataloader("val", shuffle=True), model, thresholds, mask_index=index, num=num)

plt.figure(dpi=150)
met = {k: results[k][2]["xy_max_error"][2] for k in results}
labels = ["full"] + sorted(met.keys() - {"full"}, key=lambda x: met[x])
plt.bar(np.arange(len(labels))+1, [met[k]/met["full"] for k in labels], label="sequential");
# plt.gca().set_yscale('log')
plt.gca().set_xticks(np.arange(len(labels))+1)
plt.gca().set_xticklabels(labels, fontsize=15, rotation=90)
plt.ylabel('Relative drop of accuracy')
plt.ylim([0.4, 1.2])

for k in results:
    print(results[k][2]["xy_max_error"], k)